# Gold Layer - Business Aggregations
## FinchMart Sales ETL Pipeline

This notebook creates optimized, aggregated tables for Power BI consumption:
- Daily sales aggregations
- Store performance metrics
- Top products by revenue
- Customer spending behavior

**Architecture Decision:** Using Z-Ordering and partitioning strategies to optimize query performance for analytical workloads.

In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, sum as _sum, avg, count, max as _max, min as _min,
    date_format, to_date, row_number, round as spark_round, current_timestamp
)
from pyspark.sql.window import Window
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
import os

In [ ]:
# Initialize Spark Session with Delta Lake support
builder = SparkSession.builder \
    .appName("FinchMart-Gold-Layer") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

print(f"Spark Version: {spark.version}")

In [ ]:
# Define paths
BASE_PATH = "/home/ubuntu/dataengineer-transformations-python/finchmart_sales_etl"
SILVER_PATH = f"{BASE_PATH}/data/silver/sales_transactions_clean"
GOLD_DAILY_PATH = f"{BASE_PATH}/data/gold/daily_sales"
GOLD_STORE_PATH = f"{BASE_PATH}/data/gold/store_performance"
GOLD_PRODUCTS_PATH = f"{BASE_PATH}/data/gold/top_products"
GOLD_CUSTOMERS_PATH = f"{BASE_PATH}/data/gold/customer_spending"

print(f"Silver Layer Path: {SILVER_PATH}")
print(f"Gold Layer Paths configured")

In [ ]:
# Read Silver layer data
silver_df = spark.read.format("delta").load(SILVER_PATH)

print(f"Silver layer records: {silver_df.count()}")
silver_df.printSchema()

## 1. Daily Sales Aggregation
Total sales per day across all stores and categories

In [ ]:
# Create daily sales aggregation
daily_sales_df = silver_df \
    .withColumn("transaction_date", to_date(col("transaction_timestamp"))) \
    .groupBy("transaction_date") \
    .agg(
        _sum("total_amount").alias("total_sales"),
        count("transaction_id").alias("transaction_count"),
        _sum("quantity").alias("total_items_sold"),
        avg("total_amount").alias("avg_transaction_value"),
        count(col("customer_id").distinct()).alias("unique_customers")
    ) \
    .withColumn("total_sales", spark_round(col("total_sales"), 2)) \
    .withColumn("avg_transaction_value", spark_round(col("avg_transaction_value"), 2)) \
    .withColumn("updated_at", current_timestamp()) \
    .orderBy("transaction_date")

print("Daily Sales Aggregation:")
daily_sales_df.show(10, truncate=False)

# Write to Gold layer with partitioning by date
daily_sales_df.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("transaction_date") \
    .save(GOLD_DAILY_PATH)

print(f"Daily sales data written to {GOLD_DAILY_PATH}")

## 2. Store Performance Aggregation
Total sales per store with daily breakdown

In [ ]:
# Create store performance aggregation
store_performance_df = silver_df \
    .withColumn("transaction_date", to_date(col("transaction_timestamp"))) \
    .groupBy("transaction_date", "store_location") \
    .agg(
        _sum("total_amount").alias("total_sales"),
        count("transaction_id").alias("transaction_count"),
        _sum("quantity").alias("total_items_sold"),
        avg("total_amount").alias("avg_transaction_value"),
        count(col("customer_id").distinct()).alias("unique_customers")
    ) \
    .withColumn("total_sales", spark_round(col("total_sales"), 2)) \
    .withColumn("avg_transaction_value", spark_round(col("avg_transaction_value"), 2)) \
    .withColumn("updated_at", current_timestamp()) \
    .orderBy("transaction_date", col("total_sales").desc())

print("Store Performance Aggregation:")
store_performance_df.show(20, truncate=False)

# Write to Gold layer with partitioning
store_performance_df.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("transaction_date") \
    .save(GOLD_STORE_PATH)

print(f"Store performance data written to {GOLD_STORE_PATH}")

## 3. Top Products by Revenue
Top 5 products by revenue with daily breakdown

In [ ]:
# Create top products aggregation
products_df = silver_df \
    .withColumn("transaction_date", to_date(col("transaction_timestamp"))) \
    .groupBy("transaction_date", "product_id", "product_name", "product_category") \
    .agg(
        _sum("total_amount").alias("total_revenue"),
        _sum("quantity").alias("total_quantity_sold"),
        count("transaction_id").alias("transaction_count"),
        avg("transaction_price").alias("avg_price")
    ) \
    .withColumn("total_revenue", spark_round(col("total_revenue"), 2)) \
    .withColumn("avg_price", spark_round(col("avg_price"), 2))

# Rank products by revenue within each day
window_spec = Window.partitionBy("transaction_date").orderBy(col("total_revenue").desc())

top_products_df = products_df \
    .withColumn("revenue_rank", row_number().over(window_spec)) \
    .filter(col("revenue_rank") <= 5) \
    .withColumn("updated_at", current_timestamp()) \
    .orderBy("transaction_date", "revenue_rank")

print("Top 5 Products by Revenue (per day):")
top_products_df.show(20, truncate=False)

# Write to Gold layer
top_products_df.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("transaction_date") \
    .save(GOLD_PRODUCTS_PATH)

print(f"Top products data written to {GOLD_PRODUCTS_PATH}")

## 4. Customer Spending Behavior
Average transaction value per customer with spending patterns

In [ ]:
# Create customer spending aggregation
customer_spending_df = silver_df \
    .groupBy("customer_id") \
    .agg(
        _sum("total_amount").alias("total_spent"),
        count("transaction_id").alias("transaction_count"),
        avg("total_amount").alias("avg_transaction_value"),
        _sum("quantity").alias("total_items_purchased"),
        _min("transaction_timestamp").alias("first_purchase_date"),
        _max("transaction_timestamp").alias("last_purchase_date")
    ) \
    .withColumn("total_spent", spark_round(col("total_spent"), 2)) \
    .withColumn("avg_transaction_value", spark_round(col("avg_transaction_value"), 2)) \
    .withColumn("updated_at", current_timestamp()) \
    .orderBy(col("total_spent").desc())

print("Customer Spending Behavior:")
customer_spending_df.show(20, truncate=False)

# Statistical summary
print("\nCustomer Spending Statistics:")
customer_spending_df.select(
    avg("total_spent").alias("avg_customer_lifetime_value"),
    _max("total_spent").alias("max_customer_spent"),
    _min("total_spent").alias("min_customer_spent"),
    avg("transaction_count").alias("avg_transactions_per_customer")
).show(truncate=False)

# Write to Gold layer
customer_spending_df.write \
    .format("delta") \
    .mode("overwrite") \
    .save(GOLD_CUSTOMERS_PATH)

print(f"Customer spending data written to {GOLD_CUSTOMERS_PATH}")

## 5. Performance Optimization
Apply Z-Ordering and optimize Delta tables for query performance

In [ ]:
# Optimize Gold layer tables with Z-Ordering
# Z-Ordering co-locates related information in the same set of files
# This improves query performance by reducing the amount of data read

print("Optimizing Daily Sales table...")
spark.sql(f"OPTIMIZE delta.`{GOLD_DAILY_PATH}` ZORDER BY (transaction_date)")

print("Optimizing Store Performance table...")
spark.sql(f"OPTIMIZE delta.`{GOLD_STORE_PATH}` ZORDER BY (transaction_date, store_location)")

print("Optimizing Top Products table...")
spark.sql(f"OPTIMIZE delta.`{GOLD_PRODUCTS_PATH}` ZORDER BY (transaction_date, revenue_rank)")

print("Optimizing Customer Spending table...")
spark.sql(f"OPTIMIZE delta.`{GOLD_CUSTOMERS_PATH}` ZORDER BY (customer_id)")

print("\nAll Gold layer tables optimized successfully!")

In [ ]:
# Vacuum old files (clean up old versions)
# Note: In production, set retention period appropriately (default 7 days)
print("Vacuuming Delta tables to remove old files...")

spark.sql(f"VACUUM delta.`{GOLD_DAILY_PATH}` RETAIN 0 HOURS")
spark.sql(f"VACUUM delta.`{GOLD_STORE_PATH}` RETAIN 0 HOURS")
spark.sql(f"VACUUM delta.`{GOLD_PRODUCTS_PATH}` RETAIN 0 HOURS")
spark.sql(f"VACUUM delta.`{GOLD_CUSTOMERS_PATH}` RETAIN 0 HOURS")

print("Vacuum completed")

## 6. Export Data for Power BI
Export aggregated data to CSV/Parquet for Power BI consumption

In [ ]:
# Export Gold layer tables to CSV for Power BI
EXPORT_PATH = f"{BASE_PATH}/data/gold/powerbi_export"

print("Exporting data for Power BI...")

# Export Daily Sales
daily_sales_df.coalesce(1).write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save(f"{EXPORT_PATH}/daily_sales")

# Export Store Performance
store_performance_df.coalesce(1).write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save(f"{EXPORT_PATH}/store_performance")

# Export Top Products
top_products_df.coalesce(1).write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save(f"{EXPORT_PATH}/top_products")

# Export Customer Spending
customer_spending_df.coalesce(1).write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save(f"{EXPORT_PATH}/customer_spending")

# Also export Silver layer for detailed analysis
silver_df.coalesce(1).write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save(f"{EXPORT_PATH}/transactions_detail")

print(f"All data exported to {EXPORT_PATH}")

## Summary

**Gold Layer Aggregation Complete:**
- Daily sales aggregations created with key metrics
- Store performance analyzed with daily breakdown
- Top 5 products by revenue identified per day
- Customer spending behavior analyzed
- Tables optimized with Z-Ordering for query performance
- Data partitioned by date for efficient filtering
- Data exported to CSV for Power BI consumption

**Partitioning Strategy:**
- Daily sales, store performance, and top products partitioned by `transaction_date`
- Enables efficient date-range queries
- Reduces data scanning for time-series analysis

**Z-Ordering Strategy:**
- Applied on frequently filtered columns
- Improves query performance by 2-10x for typical analytical queries
- Co-locates related data for better data skipping